In [1]:
import torch,json,os,importlib

/home/kiochy/kcc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download the pretrained models from https://zenodo.org/record/4282667 and extract the models directories into the "pretrained_models" directory.



In [2]:
os.listdir("pretrained_models/CPJKU_DCASE20T1B_pretrained/")

['ensem3', 'ensem2', 'sub2', 'sub1', 'sub3', 'ensem1']

# Select the pretrained model path

In [3]:
#model_path="pretrained_models/sub1/"
model_path="pretrained_models/CPJKU_DCASE20T1B_pretrained/sub2/"
#model_path="pretrained_models/sub3/"

# Loading the model specified by model_path

In [4]:
os.listdir(model_path)

['eval_predictions_probs.pth',
 'dataset_config.json',
 'model_from_training_state_dict.pth',
 'eval_predictions.csv',
 'model_state_dict.pth',
 'description.txt',
 'model_config.json']

### Loading the model config from json

In [5]:
with open(model_path+'model_config.json',"r")as f:
    model_config=json.load(f)


### Making an instance

In [6]:
module = importlib.import_module('models.{}'.format(model_config['arch']))
Network = getattr(module, 'Network')
model=Network(model_config)

n_blocks_per_stage is specified ignoring the depth param, nc=None



Warning Pruning only 1x1 CONVS

0 0.05892556509887896 0.408248290463863
1 0.05892556509887896 0.408248290463863
2 0.05892556509887896 0.408248290463863
3 0.1767766952966369 0.408248290463863
4 0.125 0.408248290463863
5 0.125 0.408248290463863


### Loading the pretrained weights

In [7]:
state_dict=torch.load(model_path+"/model_state_dict.pth")


In [8]:
# if the weights are float16 cast the model
if [v for k,v in state_dict.items()][0].dtype==torch.float16:
    model=model.half()
    print("model in float16")
model.load_state_dict(state_dict)


model in float16


<All keys matched successfully>

In [9]:
device = torch.device('cuda:0')
torch.cuda.is_available()

### Parameters count


In [ ]:
def count_non_zero_params_nonbn(model):
    sum_params = 0
    sum_non_zero= 0 
    desc=""
    def calc_params(model):
        nonlocal desc, sum_params, sum_non_zero
        skip="count"
        if "batchnorm" in  type(model).__name__.lower():
            skip="skip"
        for k,p in model.named_parameters(recurse=False):
            if p.requires_grad:
                nonzero = (p != 0).sum()
                total= p.numel()
                desc += f"type {type(model).__name__}, {k},  {total}, {nonzero}, {p.dtype}, {skip} " + "\n"
                if skip!="skip":
                    sum_params+=total
                    sum_non_zero+=nonzero
    model.apply(calc_params)       
    return desc,sum_params,sum_non_zero

In [ ]:
desc,sum_params,sum_non_zero=count_non_zero_params_nonbn(model)
print("Total: ",sum_params, "Non-Zero:",sum_non_zero)
print(desc)

RuntimeError: "ne_cpu" not implemented for 'Half'

# Loading DataSet

In [ ]:
import datasets
# the eval json config dataset specifies that the norms should be calculated from all the training clips. 
with open('configs/datasets/dcase2020b_eval.json',"r")as f:
    dataset_config=json.load(f)
datasetm=datasets.DatasetsManager(dataset_config['audiodataset'])

In [ ]:
trainds=datasetm.get_train_set()
testds=datasetm.get_train_set()
evalds=datasetm.get_sub_dataset()

loading dataset from 'd20t1b'
normalizing dataset
normalized train!
attempting to load x from cache at datasets/cached_datasets/d20t1b/tr_mean_676d70_f1_apd18_stereo_obj.pt...
loaded datasets/cached_datasets/d20t1b/tr_mean_676d70_f1_apd18_stereo_obj.pt from cache in 0.0005588531494140625 
attempting to load x from cache at datasets/cached_datasets/d20t1b/tr_std_676d70_f1_apd18_stereo_obj.pt...
loaded datasets/cached_datasets/d20t1b/tr_std_676d70_f1_apd18_stereo_obj.pt from cache in 0.0005135536193847656 
normalized train!
normalized test!
normalized SUB dataset!
loading dataset from 'd20t1b_sub'


## Sanity checks

### dev train

In [ ]:
all_y=0
corr_y=0
model.eval()
model.cuda()
half_mode=True
data_loader = torch.utils.data.DataLoader(trainds,
                                          batch_size=1,
                                          shuffle=False)
with torch.no_grad():
    for i,(x,idx,y) in enumerate(data_loader):
        if i%10==0:
            print(i,"/",len(data_loader),end="\r")
        x=x.cuda()
        if half_mode:
            x=x.half()
        out=model(x)
        all_y+=1
        corr_y+=(torch.argmax(out).cpu()==y).sum().item()
print("all ", all_y," correct ",corr_y," ratio=",corr_y*1./all_y) 
    

x: torch.Size([1, 2, 256, 431])
in_c: torch.Size([1, 64, 127, 215])
stage1: torch.Size([1, 64, 15, 26])
stage2: torch.Size([1, 128, 15, 26])
stage3: torch.Size([1, 128, 15, 26])
feed_forward: torch.Size([1, 3, 1, 1])
logit: torch.Size([1, 3])
all  14400  correct  14400  ratio= 1.0


# eval

In [ ]:
all_y=0
corr_y=0
model.eval()
model.cuda()
half_mode=True
data_loader = torch.utils.data.DataLoader(evalds,
                                          batch_size=1,
                                          shuffle=False,num_workers=10)
preds=[]
file_names=[]
with torch.no_grad():
    for i,(x,idx,y) in enumerate(data_loader):
        if i%10==0:
            print(i,"/",len(data_loader),end="\r")
        x=x.cuda()
        if half_mode:
            x=x.half()
        out=model(x)
        all_y+=1
        file_names+=[a for a in idx]
        preds+=[out.cpu()]
print("all ", all_y," correct ",corr_y," ratio=",corr_y*1./all_y) 
    

all  8640  correct  0  ratio= 0.0


In [ ]:
pfiles,pprobs=torch.load(model_path+"eval_predictions_probs.pth")

In [ ]:
pfiles==file_names

True

In [ ]:
print("matching",(torch.argmax(pprobs.float(),1)== torch.argmax(torch.cat(preds,0).float(),1) ).sum(),"out of " ,pprobs.shape[0])


matching tensor(8640) out of  8640
